In [2]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
from pythainlp.tag import tag_provinces


In [29]:
def crawl(url, depth,visited):
    if depth < 2 :
        visited.add(url)
        headers = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.135 Safari/537.36 Edge/12.246"}
        #time.sleep(0.25)
        response = requests.get(url,headers=headers)
        try:
            soup = BeautifulSoup(response.text, 'html.parser')
        except:
            soup = BeautifulSoup(response.text, 'lxml')
        links = soup.find_all('a')
        links = [link.get('href') for link in links if link.get('href') and not link.get('href').startswith('#')]
        links = [urljoin(url, link) for link in links if link]

        for link in links:
            if link not in visited:
                visited.add(link)
                #if link.startswith(url):
                crawl(link,depth=depth+1, visited=visited)
    return visited
base_url = 'https://www.thairath.co.th/news'
website_dict3 = crawl(base_url, 0, set())
print(website_dict3)
print(len(website_dict3))

{'https://9to5google.com/2023/02/10/oppo-find-n2-flip-event/', 'https://www.thairath.co.th/video/news/topnews/hotclip/721831', 'https://www.thairath.co.th/worldcupactivity', 'https://www.thairath.co.th/scoop/flashback', 'https://www.thairath.co.th/news/politic/2627466', 'https://plus.thairath.co.th/topic/everydaylife', 'https://www.thairath.co.th/news/society/2626963', 'https://www.thairath.co.th/horoscope/horoscope-seer/2591033', 'https://www.thairath.co.th/podcast/program/moohost', 'https://plus.thairath.co.th/topic/subculture', 'https://www.thairath.co.th/tags/%E0%B8%99%E0%B8%A4%E0%B8%A1%E0%B8%A5%20%E0%B8%A0%E0%B8%B4%E0%B8%8D%E0%B9%82%E0%B8%8D%E0%B8%AA%E0%B8%B4%E0%B8%99%E0%B8%A7%E0%B8%B1%E0%B8%92%E0%B8%99%E0%B9%8C', 'https://www.thairath.co.th/tv/watch/news', 'https://www.thairath.co.th/tags/%E0%B8%96%E0%B8%99%E0%B8%99%E0%B8%81%E0%B8%B2%E0%B8%8D%E0%B8%88%E0%B8%99%E0%B8%B2%E0%B8%A0%E0%B8%B4%E0%B9%80%E0%B8%A9%E0%B8%81', 'https://www.thairath.co.th/tv/program/khao_tieng_thairath', 'htt

In [30]:
def check_domain(base_url,links):
    result= set()
    for link in links :
        if link.startswith(base_url):
            result.add(link)
    return result



website_dict4 = check_domain(base_url,website_dict3)
print(len(website_dict4))

90


In [6]:
def check_not_domain(base_url,links):
    result= set()
    for link in links :
        if not link.startswith(base_url):
            result.add(link)
    return result

not_domain = check_not_domain(base_url,website_dict3)
print(not_domain)

{'https://plus.thairath.co.th/topic', 'line://msg/text/Mayhem Thalassic Ritual Asia Tour 2023 Live in Bangkok : คอนเสิร์ตหฤโหดที่พาสาวกดำดิ่งเข้าสู่ใจกลางแห่งอเวจี%0Ahttps://plus.thairath.co.th/topic/subculture/102770', 'https://plus.thairath.co.th/topic/everydaylife', 'https://plus.thairath.co.th/topic/everydaylife/events', 'https://plus.thairath.co.th/topic/spark/pednorkkala', 'https://plus.thairath.co.th/howtouse', 'https://plus.thairath.co.th/tag/ความรุนแรง', 'https://plus.thairath.co.th/topic/spark/101297', 'https://www.twitter.com/thairathplus', 'https://plus.thairath.co.th/category/global-affairs', 'https://plus.thairath.co.th/tag/อาวุธปืน', 'https://plus.thairath.co.th/topic/spark/opinion', 'https://plus.thairath.co.th/topic/speak/102696', 'https://plus.thairath.co.th/topic/speak/cyberandbeyond', 'https://hondasafety.thaihonda.co.th/', 'https://plus.thairath.co.th/topic/subculture', 'https://plus.thairath.co.th/topic/speak/102663', 'http://m.me/HondaSafetyThailandHeadquarter', 

In [7]:
target_links= {'https://www.thairath.co.th':0,'https://news.kapook.com':0}
def check_ref(base,links):
    for i in links:
        for j in base:
            if i.startswith(j):
                target_links[base_url]+=1
    return target_links

ref=check_ref(target_links,not_domain)
print(ref)
                

{'https://www.thairath.co.th': 0, 'https://news.kapook.com': 0}


In [8]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<30:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        self.sentence_result = unicodedata.normalize("NFKD", self.sentence_result)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = self.get_tokenize()
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=10)
        return self.summarize_result
    def get_tokenize(self):
        self.word_token = word_tokenize(self.sentence, engine="newmm")
        return self.word_token
    def get_location(self):
        self.location_Result = self.location()
        self.location_counts = {}
        for location, _ in self.location_Result:
            if location in self.location_counts:
                self.location_counts[location] += 1
            else:
                self.location_counts[location] = 1
        return self.location_counts
    def location(self):
        self.data = self.get_tokenize()
        self.location_value = tag_provinces(self.data)
        self.Result_location = filtered_input = [entry for entry in self.location_value if entry[1] == 'B-LOCATION']
        return self.Result_location

In [31]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  title_tag = soup.find('title').text
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  try:
    location = max(thai_nlp.get_location().keys())
  except:
    location = 'NONE'
  
  if  title_tag == None:
    title_tag = summarize_article[0]
    
  return p_tag, title_tag, keyword,location

def insert_data_to_db(website, p_tag, title, keyword,location):
  
  for i in keyword:
    conn.execute("INSERT INTO DATA (WEBSITE, BODY, TITLE, KEYWORD, WORD_FREQUENCY ,LOCATION) VALUES (?, ?, ?, ?, ?, ?)", (website, p_tag, title, i, keyword[i],location))
    conn.commit()
    print(f'For website {website}')
    print(f'For the p tags is:{p_tag}') 
    print(f'For the title tag is: {title}')
    print(f'For the keyword is:{i}')
    print(f'Forthe word frequency is:{keyword[i]}')
    print(f'For the location is:{location}')

 

# Iterate through the websites
conn = sqlite3.connect('scraped_data2.db')
conn.execute('''CREATE TABLE DATA
             (ID INTEGER PRIMARY KEY AUTOINCREMENT,
             WEBSITE STRING NOT NULL,
             BODY TEXT NOT NULL,
             TITLE TEXT NOT NULL,
             KEYWORD TEXT NOT NULL,
             WORD_FREQUENCY INT NOT NULL,
             LOCATION TEXT );''')
j=0
for website in website_dict4:
  p_tag, title, keyword,location = scrape_tags(website)
  insert_data_to_db(website, p_tag, title, keyword,location)
  j+=1
  print(j)
       


conn.close()





For website https://www.thairath.co.th/news/politic/2627366
For the p tags is:"คุณหญิงสุดารัตน์" ยัน เรื่องเท็จ "ไทยสร้างไทย" ควบรวม รทสช. ลั่น ไม่มีทางนําพรรคไปรวมกับเผด็จการเด็ดขาด พร้อมส่งสาร 3 ประการ "ไม่มีทางเป็นนั่งร้านเผด็จการ-เจตนารมณ์แก้รธน.-ชวนคนไทยไปเลือกตั้ง"  ดิฉันขอเรียนยืนยันว่า เรื่องนี้เป็นเรื่องเท็จทั้งสิ้น1) ดิฉัน “ไม่มีวันที่ดิฉันจะนําพรรคไทยสร้างไทย ที่ยืนหยัดรักษาประชาธิปไตย ไปรวมกับพรรคการเมืองอื่นใดที่อยู่ฝ่ายเผด็จการ” อย่างเด็ดขาด2) พรรคไทยสร้างไทยยืนยันเดินหน้าสร้างพรรคให้เป็นสถาบันการเมืองของประชาชน แม้จะมีอุปสรรคมากมาย ดิฉันขอสู้ไม่ถอยค่ะดิฉัน และพรรคไทยสร้างไทยขอเดินหน้าทํางานอย่างเต็มที่ และพร้อมเข้าสู่การเลือกตั้ง ถึงแม้ว่า เราจะเป็นพรรคการเมืองน้องใหม่ ต้องต่อสู้กับอํานาจเงิน อํานาจรัฐ ที่จะถูกใช้อย่างเต็มที่ในการเลือกตั้งครั้งนี้ แต่หัวใจของเราแข็งแกร่ง และมั่นคงกับจุดมุ่งหมายของเราที่ “ต้องการสร้างและส่งมอบประเทศไทยที่ดีกว่านี้ให้กับคนรุ่นต่อไป”ดิฉันขอเดินหน้าทําภารกิจสุดท้ายในการสร้างพรรคไทยสร้างไทยให้เป็นทาง “รอด” ของประเทศ จากวิกฤติความขัดแย้งการเมื